# Лабораторная работа: Мета-обучение
## Шаги 3-4: Оценка алгоритмов и построение мета-набора

**Цель:** Оценить базовые алгоритмы на каждом датасете и определить лучший

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, StratifiedKFold
from tqdm import tqdm

from src.algorithms.base_learners import BaseLearners
from src.algorithms.evaluation import AlgorithmEvaluator
from src.data.collector import OpenMLCollector
from src.visualization.plots import MetaVisualizer

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
# Загружаем список датасетов
datasets_df = pd.read_csv('../data/raw/dataset_list.csv')
print(f"Загружено {len(datasets_df)} датасетов")

# Загружаем мета-признаки (если уже есть)
try:
    meta_df = pd.read_csv('../data/processed/meta_features_raw.csv')
    print(f"Загружены мета-признаки для {len(meta_df)} датасетов")
except:
    print("Мета-признаки не найдены, будут извлечены позже")

### 3.1 Выбор алгоритмов для оценки

In [ ]:
# Выбираем 3 алгоритма с разной природой
learners = BaseLearners(['LogisticRegression', 'RandomForest', 'KNN'])

print("Выбранные алгоритмы:")
for name, algo in learners.get_all_algorithms().items():
    print(f"  - {name}: {algo.__class__.__name__}")

# Создаем оценщик
evaluator = AlgorithmEvaluator(
    learners.get_all_algorithms(),
    cv_folds=5,
    scoring='balanced_accuracy'
)

visualizer = MetaVisualizer(save_dir='../results/visualizations')

### 3.2 Тестирование на примере

In [ ]:
# Загружаем тестовый датасет
collector = OpenMLCollector(use_cache=True)
X, y = collector.download_dataset(61)  # Iris

print(f"Тестовый датасет: {X.shape[0]} объектов, {X.shape[1]} признаков, {len(np.unique(y))} классов")

# Оцениваем
scores = evaluator.evaluate_all(X, y)
best_algo, best_score = evaluator.get_best_algorithm(X, y)

print("\nРезультаты оценки:")
for algo, score in scores.items():
    print(f"  {algo}: {score:.4f}")
print(f"\nЛучший алгоритм: {best_algo} с точностью {best_score:.4f}")

### 3.3 Массовая оценка алгоритмов

In [ ]:
# Для демонстрации возьмем первые 10 датасетов
# В реальном проекте будет 350
sample_datasets = datasets_df.head(10)

results = []

for idx, row in tqdm(sample_datasets.iterrows(), total=len(sample_datasets)):
    dataset_id = row['did']
    
    try:
        X, y = collector.download_dataset(dataset_id)
        
        if X is not None:
            scores = evaluator.evaluate_all(X, y)
            
            result = {
                'dataset_id': dataset_id,
                'dataset_name': row['name'],
                'best_algorithm': max(scores, key=scores.get),
                **{f'score_{k}': v for k, v in scores.items()}
            }
            results.append(result)
    except Exception as e:
        print(f"Ошибка в датасете {dataset_id}: {e}")

results_df = pd.DataFrame(results)
print(f"\nОценено {len(results_df)} датасетов")
results_df

In [ ]:
# Анализ результатов
print("\nРаспределение лучших алгоритмов:")
best_counts = results_df['best_algorithm'].value_counts()
print(best_counts)

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Распределение лучших алгоритмов
best_counts.plot(kind='bar', ax=axes[0], color='coral', alpha=0.7)
axes[0].set_xlabel('Алгоритм')
axes[0].set_ylabel('Количество датасетов')
axes[0].set_title('Распределение лучших алгоритмов')
axes[0].tick_params(axis='x', rotation=45)

# Производительность алгоритмов
score_cols = [c for c in results_df.columns if c.startswith('score_')]
score_data = []
for col in score_cols:
    for score in results_df[col].dropna():
        score_data.append({'Algorithm': col.replace('score_', ''), 'Score': score})

score_df = pd.DataFrame(score_data)
sns.boxplot(data=score_df, x='Algorithm', y='Score', ax=axes[1])
axes[1].set_title('Распределение производительности алгоритмов')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 3.4 Построение мета-набора данных

In [ ]:
# Объединяем мета-признаки с результатами
if 'meta_df' in locals() and not meta_df.empty and not results_df.empty:
    meta_dataset = pd.merge(meta_df, results_df[['dataset_id', 'best_algorithm']], on='dataset_id')
    print(f"Мета-набор данных: {meta_dataset.shape[0]} строк, {meta_dataset.shape[1]} колонок")
    
    # Сохраняем
    meta_dataset.to_csv('../results/meta_dataset/meta_dataset_complete.csv', index=False)
    print("Мета-набор сохранен в ../results/meta_dataset/meta_dataset_complete.csv")
else:
    print("Недостаточно данных для построения мета-набора")

### 3.5 Выводы по шагам 3-4

In [ ]:
print("="*60)
print("ВЫВОДЫ ПО ШАГАМ 3-4: ОЦЕНКА АЛГОРИТМОВ")
print("="*60)
print(f"""
1. ВЫБРАНЫ АЛГОРИТМЫ:
   - Logistic Regression (линейная модель)
   - Random Forest (ансамблевая модель)
   - KNN (модель на основе расстояний)

2. МЕТРИКА ОЦЕНКИ: balanced_accuracy (устойчива к дисбалансу классов)

3. ВАЛИДАЦИЯ: 5-кратная стратифицированная кросс-валидация

4. РЕЗУЛЬТАТЫ:
   - Оценено {len(results_df)} датасетов
   - Распределение лучших алгоритмов:
{best_counts.to_string()}

5. МЕТА-НАБОР ДАННЫХ СОЗДАН:
   - Каждая строка: датасет
   - Признаки: мета-признаки
   - Целевая переменная: лучший алгоритм
""")